In [24]:
import requests
import json
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [26]:
large = pd.read_excel('/Users/kimmh/Desktop/서데캠/data/한국행정구역분류_2021.10.1.기준.xlsx', sheet_name=3, skiprows=1)

large = large[large['시도']=='서울특별시']
large = large[large['시군구']!='서울특별시']
large = large[['행정동(행정기관명)', '법정동', '행정기관코드', '법정동코드']]
large.columns = ['행정동', '법정동', '행정동코드', '법정동코드']

large.head()

,행정동,법정동,행정동코드,법정동코드
1,종로구,종로구,1111000000,1111000000
2,청운효자동,청운동,1111051500,1111010100
3,청운효자동,신교동,1111051500,1111010200
4,청운효자동,궁정동,1111051500,1111010300
5,청운효자동,효자동,1111051500,1111010400


#### 25개 구 제외

In [27]:
large = large[-large['행정동'].str.endswith('구')]

print(large.shape)
large.head()

(741, 4)


,행정동,법정동,행정동코드,법정동코드
2,청운효자동,청운동,1111051500,1111010100
3,청운효자동,신교동,1111051500,1111010200
4,청운효자동,궁정동,1111051500,1111010300
5,청운효자동,효자동,1111051500,1111010400
6,청운효자동,창성동,1111051500,1111010500


In [31]:
# large.to_csv('/Users/kimmh/Desktop/서데캠/data/행정_법정_매칭_new.txt', sep=',', index=False)

---

In [106]:
url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1086228&y=37.4012191" # 여기 x, y값에 위경도 대입하면 될듯 

headers = {"Authorization": "KakaoAK a12d3a71c08429eed2789c17a0c354ac"} # 이건 내 api 발급받은거 

In [107]:
result = json.loads(str(requests.get(url, headers=headers).text))
result['documents']
# H : 행정동 의미

[{'region_type': 'B',
  'code': '4113510900',
  'address_name': '경기도 성남시 분당구 삼평동',
  'region_1depth_name': '경기도',
  'region_2depth_name': '성남시 분당구',
  'region_3depth_name': '삼평동',
  'region_4depth_name': '',
  'x': 127.1163593869371,
  'y': 37.40612091848614},
 {'region_type': 'H',
  'code': '4113565500',
  'address_name': '경기도 성남시 분당구 삼평동',
  'region_1depth_name': '경기도',
  'region_2depth_name': '성남시 분당구',
  'region_3depth_name': '삼평동',
  'region_4depth_name': '',
  'x': 127.1163593869371,
  'y': 37.40612091848614}]

In [4]:
result['documents'][1]['region_3depth_name']

'삼평동'

### 좌표 입력하여 행정동 추출하는 함수

In [5]:
def getDong(long, lat):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(str(long), str(lat)) 
    # 경도 / 위도 formatting 
    headers = {"Authorization": "KakaoAK a12d3a71c08429eed2789c17a0c354ac"} # 내 인증키 
    result = json.loads(str(requests.get(url, headers=headers).text))
    dong = result['documents'][1]['region_3depth_name']
    return dong

In [6]:
getDong(127.11786562005, 37.528682790758)  # 풍납동 현대아파트 위도 경도 

'풍납2동'

In [7]:
charger = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/api_charger_processed.txt')
print(charger.shape)
charger.head()

(3548, 6)


,addr,lat,lng,법정동,cnt,법정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800


In [8]:
longlat = charger[['lng', 'lat']]
print(len(longlat))
longlat.head()

3548


,lng,lat
0,127.043759,37.503534
1,127.023600,37.509900
2,127.032163,37.490051
3,127.029404,37.495268
4,127.026766,37.508248


In [9]:
### 시간 매우 오래 걸림 ### 

dong = []

for i in longlat.index: 
    dong.append(getDong(longlat['lng'][i], longlat['lat'][i]))
    
longlat['행정동'] = dong
longlat.head()

<ipython-input-9-744a73eae52e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  longlat['행정동'] = dong


,lng,lat,행정동
0,127.043759,37.503534,역삼1동
1,127.023600,37.509900,논현1동
2,127.032163,37.490051,역삼1동
3,127.029404,37.495268,역삼1동
4,127.026766,37.508248,논현1동


#### longlat 데이터 확인

In [10]:
print(longlat.shape)
longlat.head(n=10)

(3548, 3)


,lng,lat,행정동
0,127.043759,37.503534,역삼1동
1,127.023600,37.509900,논현1동
2,127.032163,37.490051,역삼1동
3,127.029404,37.495268,역삼1동
4,127.026766,37.508248,논현1동
5,127.022903,37.515043,논현1동
6,127.031082,37.497549,역삼1동
7,127.079280,37.494910,일원2동
8,127.081799,37.494966,일원2동
9,127.048510,37.481510,개포4동


In [11]:
longlat.isna().sum()

lng    0
lat    0
행정동    0
dtype: int64

### 위/경도 - 행정동 매핑 데이터 저장

In [12]:
# longlat.to_csv('/Users/kimmh/Desktop/서데캠/data/위경도_행정동.txt', sep=',', index=False)

---

---

---

## 전기차 충전소 현황 데이터에 행정동 열 추가 (위도/경도 기준)

In [186]:
charger = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/api_charger_processed.txt')
print(charger.shape)
charger.head()

(3548, 6)


,addr,lat,lng,법정동,cnt,법정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800


In [187]:
longlat = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/위경도_행정동.txt')
print(longlat.shape)
longlat.head()

(3548, 3)


,lng,lat,행정동
0,127.043759,37.503534,역삼1동
1,127.023600,37.509900,논현1동
2,127.032163,37.490051,역삼1동
3,127.029404,37.495268,역삼1동
4,127.026766,37.508248,논현1동


### 결측치 19개 > 위/경도가 잘못된 경우

In [188]:
charger['행정동'] = longlat['행정동']
charger.head()

,addr,lat,lng,법정동,cnt,법정동코드,행정동
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100,역삼1동
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800,논현1동
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100,역삼1동
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100,역삼1동
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800,논현1동


In [189]:
na1 = charger[charger['행정동'].isna()] 
na1

,addr,lat,lng,법정동,cnt,법정동코드,행정동
791,서울특별시 광진구 능동로 19,33.0,124.0,자양동,4,1121510500,NaN
884,서울특별시 구로구 경인로8길 64,33.0,124.0,오류동,10,1153010800,NaN
956,서울특별시 구로구 디지털로34길 27,33.0,124.0,구로동,2,1153010200,NaN
1190,서울특별시 노원구 동일로 1127,33.0,124.0,공릉동,3,1135010300,NaN
1310,서울특별시 도봉구 도봉로136가길 73,33.0,124.0,창동,8,1132010700,NaN
1314,서울특별시 도봉구 도봉로169길 31,33.0,124.0,도봉동,3,1132010800,NaN
1381,서울특별시 도봉구 해등로16길 12,33.0,124.0,창동,5,1132010700,NaN
1402,서울특별시 동대문구 망우로 95,33.0,124.0,휘경동,8,1123010900,NaN
1497,서울특별시 동대문구 황물로15길 25,33.0,124.0,답십리동,6,1123010500,NaN
1581,서울특별시 동작구 서달로 91,33.0,124.0,흑석동,10,1159010500,NaN


In [190]:
url = 'https://dapi.kakao.com/v2/local/search/address.json?query=서울 마포구 모래내로1길 20'
headers = {"Authorization": "KakaoAK a12d3a71c08429eed2789c17a0c354ac"} 
result = json.loads(str(requests.get(url, headers=headers).text))

result['documents'][0]['address']['region_3depth_h_name']

'성산2동'

In [191]:
def getHang(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr
    headers = {"Authorization": "KakaoAK a12d3a71c08429eed2789c17a0c354ac"}
    result = json.loads(str(requests.get(url, headers=headers).text))
    return result['documents'][0]['address']['region_3depth_h_name']

getHang(charger['addr'][0])

'역삼1동'

In [192]:
for i in na1.index: 
    charger['행정동'][i] = getHang(charger['addr'][i])
    
charger.head()

,addr,lat,lng,법정동,cnt,법정동코드,행정동
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100,역삼1동
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800,논현1동
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100,역삼1동
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100,역삼1동
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800,논현1동


In [193]:
charger.isna().sum()

addr     0
lat      0
lng      0
법정동      0
cnt      0
법정동코드    0
행정동      0
dtype: int64

### 매칭 데이터 불러오기

In [203]:
match = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/행정_법정_매칭_new.txt')
match = match[['행정동코드', '행정동']]

print(match.shape)
match.head()

(741, 2)


,행정동코드,행정동
0,1111051500,청운효자동
1,1111051500,청운효자동
2,1111051500,청운효자동
3,1111051500,청운효자동
4,1111051500,청운효자동


### 풍납제2동 이따구로 되어있는 거 풍납2동으로 바꾸기

In [205]:
je = match[match['행정동'].str.contains('제')]
je.head()

,행정동코드,행정동
88,1111067000,창신제1동
89,1111068000,창신제2동
90,1111069000,창신제3동
91,1111070000,숭인제1동
92,1111071000,숭인제2동


#### 홍제동 빼고는 모두 제1동 이런 식 >> 제거 후 홍제동만 수정

In [206]:
je = match['행정동'].str.replace('제', repl = r'')

In [207]:
match['행정동2'] = je 
match.head()

,행정동코드,행정동,행정동2
0,1111051500,청운효자동,청운효자동
1,1111051500,청운효자동,청운효자동
2,1111051500,청운효자동,청운효자동
3,1111051500,청운효자동,청운효자동
4,1111051500,청운효자동,청운효자동


In [208]:
match[match['행정동2'].str.contains('제')]  # '제' 잘 제거됨, 이제 홍제동만 추가

,행정동코드,행정동,행정동2


In [209]:
hong = match[(match['행정동2']=='홍1동')|(match['행정동2']=='홍2동')|(match['행정동2']=='홍3동')]
hong

,행정동코드,행정동,행정동2
425,1141062000,홍제제1동,홍1동
426,1141064000,홍제제3동,홍3동
427,1141065500,홍제제2동,홍2동


In [210]:
match['행정동2'] = match['행정동2'].str.replace('홍1동', '홍제1동')
match['행정동2'] = match['행정동2'].str.replace('홍2동', '홍제2동')
match['행정동2'] = match['행정동2'].str.replace('홍3동', '홍제3동')

match[(match['행정동2']=='홍1동')|(match['행정동2']=='홍2동')|(match['행정동2']=='홍3동')]

,행정동코드,행정동,행정동2


#### 홍제동 잘 수정됨

In [211]:
match.drop('행정동', axis=1, inplace=True)
match.head()

,행정동코드,행정동2
0,1111051500,청운효자동
1,1111051500,청운효자동
2,1111051500,청운효자동
3,1111051500,청운효자동
4,1111051500,청운효자동


In [212]:
match.columns = ['행정동코드', '행정동']
match.head()

,행정동코드,행정동
0,1111051500,청운효자동
1,1111051500,청운효자동
2,1111051500,청운효자동
3,1111051500,청운효자동
4,1111051500,청운효자동


In [213]:
match.shape

(741, 2)

---

In [214]:
hang = match.drop_duplicates(['행정동코드', '행정동'], keep='first', ignore_index=True)

print(hang.shape)
hang.head()

(426, 2)


,행정동코드,행정동
0,1111051500,청운효자동
1,1111053000,사직동
2,1111054000,삼청동
3,1111055000,부암동
4,1111056000,평창동


In [255]:
merged = pd.merge(charger, hang, how='left', on='행정동')
print(merged.shape)
merged.head()

(3559, 8)


,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100,역삼1동,1.168064e+09
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800,논현1동,1.168052e+09
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100,역삼1동,1.168064e+09
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100,역삼1동,1.168064e+09
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800,논현1동,1.168052e+09


In [256]:
merged.isna().sum()

addr      0
lat       0
lng       0
법정동       0
cnt       0
법정동코드     0
행정동       0
행정동코드    25
dtype: int64

---

### 결측치 : 위/경도 변환 오류

In [257]:
na = merged[merged['행정동코드'].isna()]
na

,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
62,서울특별시 강남구 논현로51길 25,37.769477,126.770067,도곡동,2,168011800,금촌3동,NaN
660,서울특별시 강서구 허준로 47,37.500000,126.800000,가양동,3,1150010400,부천동,NaN
860,서울특별시 구로구 가마산로25길 33(구로동),37.703795,126.903946,구로동,1,1153010200,고양동,NaN
1072,서울특별시 금천구 디지털로9길 33,35.122107,126.907958,가산동,3,1154510100,봉선2동,NaN
1240,서울특별시 노원구 상계로 267,37.405561,127.423366,상계동,2,1135010500,산북면,NaN
1395,서울특별시 동대문구 고산자로 534,37.588393,127.039218,제기동,10,1123010300,제기동,NaN
1435,서울특별시 동대문구 약령중앙로 26,37.580510,127.037150,제기동,2,1123010300,제기동,NaN
1436,서울특별시 동대문구 왕산로 109,37.578569,127.035574,제기동,2,1123010300,제기동,NaN
1478,서울특별시 동대문구 정릉천동로 83,37.579786,127.034961,제기동,3,1123010300,제기동,NaN
1479,서울특별시 동대문구 정릉천동로 83,37.579787,127.034961,제기동,3,1123010300,제기동,NaN


In [258]:
merged[merged['행정동']=='제기동'] = merged[merged['행정동']=='제기동'].fillna(1123054500)
merged[merged['행정동']=='면목제3.8동'] = merged[merged['행정동']=='면목제3.8동'].fillna(1126057500)

In [259]:
data = merged 

In [260]:
na = data[data['행정동코드'].isna()]
na

,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
62,서울특별시 강남구 논현로51길 25,37.769477,126.770067,도곡동,2,168011800,금촌3동,NaN
660,서울특별시 강서구 허준로 47,37.500000,126.800000,가양동,3,1150010400,부천동,NaN
860,서울특별시 구로구 가마산로25길 33(구로동),37.703795,126.903946,구로동,1,1153010200,고양동,NaN
1072,서울특별시 금천구 디지털로9길 33,35.122107,126.907958,가산동,3,1154510100,봉선2동,NaN
1240,서울특별시 노원구 상계로 267,37.405561,127.423366,상계동,2,1135010500,산북면,NaN
1613,"서울특별시 동작구 여의대방로10길 97 (신대방동, 남부수도사업소)",37.429300,126.915400,신대방동,4,1159010900,석수1동,NaN
2277,"서울특별시 성동구 무학로12길 11 (홍익동, 청계대주파크빌)",37.340200,127.189000,홍익동,1,1120010300,모현읍,NaN
2450,서울특별시 성북구 장월로 160,38.025677,127.072628,장위동,6,1129013800,전곡읍,NaN
2752,서울특별시 양천구 목동중앙로13길 15,36.329467,127.409169,용두동,2,1123010200,용두동,NaN
2946,서울특별시 영등포구 여의동로 97,35.693199,128.456344,여의도동,2,1156011000,현풍읍,NaN


In [261]:
getHang(na['addr'][62])

'도곡1동'

In [262]:
for i in na.index: 
    data['행정동'][i] = getHang(data['addr'][i])
    
data.head()

,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100,역삼1동,1.168064e+09
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800,논현1동,1.168052e+09
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100,역삼1동,1.168064e+09
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100,역삼1동,1.168064e+09
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800,논현1동,1.168052e+09


In [263]:
data.isna().sum()

addr      0
lat       0
lng       0
법정동       0
cnt       0
법정동코드     0
행정동       0
행정동코드    12
dtype: int64

In [265]:
na = data[data['행정동코드'].isna()]
na

,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
62,서울특별시 강남구 논현로51길 25,37.769477,126.770067,도곡동,2,168011800,도곡1동,NaN
660,서울특별시 강서구 허준로 47,37.500000,126.800000,가양동,3,1150010400,가양1동,NaN
860,서울특별시 구로구 가마산로25길 33(구로동),37.703795,126.903946,구로동,1,1153010200,구로5동,NaN
1072,서울특별시 금천구 디지털로9길 33,35.122107,126.907958,가산동,3,1154510100,가산동,NaN
1240,서울특별시 노원구 상계로 267,37.405561,127.423366,상계동,2,1135010500,상계3.4동,NaN
1613,"서울특별시 동작구 여의대방로10길 97 (신대방동, 남부수도사업소)",37.429300,126.915400,신대방동,4,1159010900,신대방2동,NaN
2277,"서울특별시 성동구 무학로12길 11 (홍익동, 청계대주파크빌)",37.340200,127.189000,홍익동,1,1120010300,왕십리도선동,NaN
2450,서울특별시 성북구 장월로 160,38.025677,127.072628,장위동,6,1129013800,장위3동,NaN
2752,서울특별시 양천구 목동중앙로13길 15,36.329467,127.409169,용두동,2,1123010200,목2동,NaN
2946,서울특별시 영등포구 여의동로 97,35.693199,128.456344,여의도동,2,1156011000,여의동,NaN


In [267]:
data['행정동코드'][62] = 1168065500 
data['행정동코드'][660] = 1150060300
data['행정동코드'][860] = 1153056000
data['행정동코드'][1072] = 1154551000
data['행정동코드'][1240] = 1135066500
data['행정동코드'][1613] = 1159068000
data['행정동코드'][2277] = 1120053500
data['행정동코드'][2450] = 1129078000
data['행정동코드'][2752] = 1147052000
data['행정동코드'][2946] = 1156054000
data['행정동코드'][3140] = 1138052000
data['행정동코드'][3285] = 1111061500

In [270]:
na = data[data['행정동코드'].isna()]
na

,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드


In [271]:
data.isna().sum()

addr     0
lat      0
lng      0
법정동      0
cnt      0
법정동코드    0
행정동      0
행정동코드    0
dtype: int64

## 결측치 제거 완료!

In [274]:
print(data.shape)
data.head()

(3559, 8)


,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100,역삼1동,1.168064e+09
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800,논현1동,1.168052e+09
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100,역삼1동,1.168064e+09
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100,역삼1동,1.168064e+09
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800,논현1동,1.168052e+09


In [275]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3559 entries, 0 to 3558
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   addr    3559 non-null   object 
 1   lat     3559 non-null   float64
 2   lng     3559 non-null   float64
 3   법정동     3559 non-null   object 
 4   cnt     3559 non-null   int64  
 5   법정동코드   3559 non-null   int64  
 6   행정동     3559 non-null   object 
 7   행정동코드   3559 non-null   float64
dtypes: float64(3), int64(2), object(3)
memory usage: 379.3+ KB


In [276]:
data['행정동코드'] = data['행정동코드'].astype(int)
data.head()

,addr,lat,lng,법정동,cnt,법정동코드,행정동,행정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100,역삼1동,1168064000
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800,논현1동,1168052100
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100,역삼1동,1168064000
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100,역삼1동,1168064000
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800,논현1동,1168052100


In [277]:
# data.to_csv('/Users/kimmh/Desktop/서데캠/data/api_charger_final.txt', sep=',', index=False)

## 전기차 충전소 현황 > 행정동코드 저장